In [55]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


/Users/csuarezgurruchaga/Desktop/Digital-House/CLASE_44/dsad_2021/common
default checking
Running command `conda list`... ok
jupyterlab=2.2.6 already installed
pandas=1.1.5 already installed
bokeh=2.2.3 already installed
seaborn=0.11.0 already installed
matplotlib=3.3.2 already installed
ipywidgets=7.5.1 already installed
pytest=6.2.1 already installed
chardet=4.0.0 already installed
psutil=5.7.2 already installed
scipy=1.5.2 already installed
statsmodels=0.12.1 already installed
scikit-learn=0.23.2 already installed
xlrd=2.0.1 already installed
Running command `conda install --yes nltk=3.5.0`... ok
Solving environment: ...working... done

# All requested packages already installed.


unidecode=1.1.1 already installed
pydotplus=2.0.2 already installed
pandas-datareader=0.9.0 already installed
flask=1.1.2 already installed


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>

# Descenso gradiente

Vamos a usar el dataset de propiedades en Boston (https://www.kaggle.com/c/boston-housing) y tratar de predecir el valor de `medv` usando una regresión lineal múltiple.

Para eso, vamos a modificar la clase `MyGradientDescent` presentada en el encuentro sincrónico y usarla para entrenar un modelo de regresión lineal múltiple y uno simple.

Ayuda: 

<p style="font-size:16px;">
$h =  \beta_0 + \beta_1. X_1 + \beta_2. X_2 + \beta_3. X_3 + ... + \beta_m. X_m$
</p>

i es el índice de la fila en el dataset

<p style="font-size:16px;">
$h_i =  \beta_0 + \beta_1. X_{i1} + \beta_2. X_{i2} + \beta_3. X_{i3} + ... + \beta_m. X_{im}$
</p>    

Update: 

<p style="font-size:16px;">
$\beta_0 = \beta_0 - \alpha \frac{1}{N} \sum (h_i - y_i)$
</p>    
<p style="font-size:16px;">
$\beta_i = \beta_i - \alpha \frac{1}{N} \sum (h_i - y_i). X_i$
</p>    

Costo (error cuadrático medio): 
<p style="font-size:16px;">    
$J(\beta_0, ..., \beta_m) = \frac{1}{N} \sum_{i=1}^N (h_i - y_i)^2  $
</p>
Gradiente: 

<p style="font-size:16px;">    
$\frac{\partial J(\beta_0, ..., \beta_m)}{\partial \beta_j} = \frac{2}{N} \sum_{i=1}^N (h_i - y_i). X_{ij} $
</p>    

N es el número de observaciones o filas del dataset

Entonces 

$\beta_0 = \beta_0 - \alpha .\frac{2}{N} \sum_{i=1}^N (h_i - y_i). X_{i0}$

como $X_{i0} = 1$ queda:

$\beta_0 = \beta_0 - \alpha .\frac{2}{N} \sum_{i=1}^N (h_i - y_i)$

$\beta_1 = \beta_1 - \alpha .\frac{2}{N} \sum_{i=1}^N (h_i - y_i). X_{i1}$

$\beta_2 = \beta_2 - \alpha .\frac{2}{N} \sum_{i=1}^N (h_i - y_i). X_{i2}$

... 

$\beta_j = \beta_j - \alpha .\frac{2}{N} \sum_{i=1}^N (h_i - y_i). X_{ij}$


$\alpha$ = Learning Rate

## Imports

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

## Ejercicio 1

Leer los datos del archivo `Data/boston_data.csv` en un dataframe y construir un heatmap de correlaciones entre sus columnas

In [32]:
data = pd.read_csv('../Data/boston_data.csv')

data.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.15876,0.0,10.81,0.0,0.413,5.961,17.5,5.2873,4.0,305.0,19.2,376.94,9.88,21.7
1,0.10328,25.0,5.13,0.0,0.453,5.927,47.2,6.9320,8.0,284.0,19.7,396.90,9.22,19.6
2,0.34940,0.0,9.90,0.0,0.544,5.972,76.7,3.1025,4.0,304.0,18.4,396.24,9.97,20.3
3,2.73397,0.0,19.58,0.0,0.871,5.597,94.9,1.5257,5.0,403.0,14.7,351.85,21.45,15.4
4,0.04337,21.0,5.64,0.0,0.439,6.115,63.0,6.8147,4.0,243.0,16.8,393.97,9.43,20.5


In [33]:
sns.heatmap(data.corr());

## Ejercicio 2 

La variable target del modelo es `medv`.

Seleccionar como variables predictoras las tres variables que tengan mayor correlación (en valor absoluto) con la variable target.

Construir los conjuntos de train y test y normalizar las features.

In [34]:
abs(data.corr()['medv']).sort_values()

chas       0.141400
dis        0.264876
zn         0.355608
black      0.360071
age        0.390863
crim       0.400956
rad        0.423508
nox        0.439225
tax        0.495792
indus      0.501698
ptratio    0.506313
rm         0.683541
lstat      0.742695
medv       1.000000
Name: medv, dtype: float64

In [35]:
features = ['lstat', 'rm', 'ptratio']
X_features = data[features]
y = data['medv']

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y, random_state = 12)

In [37]:
scaler = StandardScaler()
X_train_scl = scaler.fit_transform(X_train)
X_test_scl = scaler.transform(X_test)

## Ejercicio 3

Modificar la clase `MyGradientDescent` presentada en el encuentro sincrónico para resolver ahora una regresión **múltiple** usando descenso gradiente.

```

class MyGradientDescent():
    def __init__(self, learning_rate):
        self.learning_rate = learning_rate
        self.beta1 = 0
        self.beta0 = 0
          
    def fit(self, X, y, epochs = 100):
        N = len(X)
        history = []
        
        for e in range(epochs):
            for i in range(N):
                Xi = X[i, :]
                yi = y.iloc[i]                 
                
                hi = self.beta1 * Xi + self.beta0
                f = hi - yi
                
                self.beta1 -= self.learning_rate * 2 / N * f * Xi
                self.beta0 -= self.learning_rate * 2 / N * f 

            loss = 0
            loss = mean_squared_error(y, (self.beta1 * X + self.beta0))
                                      
            if e % 100 == 0:
                print(f"Epoch: {e}, Loss: {loss})")
            
            history.append(loss)
                                      
        return history
                
    def predict(self, X):
        return self.beta1 * X + self.beta0


```

Tener en cuenta las fórmulas presentadas en la ayuda al inicio de la notebook.

In [38]:
np.repeat(0, 10)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [39]:
class MyGradientDescentMultiple():
    
    def __init__(self, learning_rate, m):
        self.learning_rate = learning_rate
        self.betas = np.repeat(0, m)
        self.beta0 = 0
          
    def fit(self, X, y, epochs = 100):
        N = len(X)
        m = X.shape[1] 
        history = []
        
        for e in range(epochs):
            gradiente_0 = 0
            gradiente = np.repeat(0, m)
            for i in range(N):
                Xi = X[i, :]
                yi = y.iloc[i]                 

                hi = np.dot(Xi, self.betas) + self.beta0    
                
                gradiente = gradiente + (hi - yi) * Xi
                gradiente_0 = gradiente_0 + (hi - yi)


            self.beta0 = self.beta0 - self.learning_rate * 2 / N * gradiente_0   
            self.betas = self.betas - self.learning_rate * 2 / N * gradiente
                    
            pred = np.dot(X, self.betas) + self.beta0
            loss = mean_squared_error(y, pred)
                                      
            if e % 100 == 0:
                print(f"Epoch: {e}, Loss: {loss})")
            
            history.append(loss)
                                      
        return history
            
    def predict(self, X):
        return np.dot(X, self.betas) + self.beta0

## Ejercicio 4

Entrenar la regresión lineal múltiple con tres variables predictoras usando la clase que definieron en el ejercicio 3.

Evaluar la performance en test mediante el error cuadrático medio.

In [40]:
model = MyGradientDescentMultiple(learning_rate = 0.01, m = 3)
history = model.fit(X_train_scl, y_train, 1000)

predictions = model.predict(X_test_scl)

Epoch: 0, Loss: 552.9706208122417)
Epoch: 100, Loss: 32.747061639398275)
Epoch: 200, Loss: 24.419506634518687)
Epoch: 300, Loss: 24.266662635172867)
Epoch: 400, Loss: 24.262934296516512)
Epoch: 500, Loss: 24.262703056067398)
Epoch: 600, Loss: 24.26267212647544)
Epoch: 700, Loss: 24.26266719726997)
Epoch: 800, Loss: 24.262666392930523)
Epoch: 900, Loss: 24.262666261234756)


In [41]:
mean_squared_error(y_test, predictions)

22.411860260524367

## Ejercicio 5

Graficar el valor de pérdida en función de las épocas

In [42]:
sns.lineplot(x = range(len(history)), y = history);

## Ejercicio 6

Usar la misma clase del ejercicio 3 para ajustar una regresión lineal simple cuya variable predictora sea `lstat` y comprobar que esta clase da el mismo resultado que `MyGradientDescent`

Graficar en un scatterplot los datos de test y los predichos por el modelo.

In [43]:
features = ['lstat']
X_features = data[features]
y = data['medv']

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y, random_state = 12)

In [45]:
scaler = StandardScaler()
X_train_scl = scaler.fit_transform(X_train)
X_test_scl = scaler.transform(X_test)

In [46]:
model_simple = MyGradientDescentMultiple(learning_rate = 0.01, m = 1)
history = model_simple.fit(X_train_scl, y_train, 1000)

predictions = model_simple.predict(X_test_scl)

Epoch: 0, Loss: 555.245046095597)
Epoch: 100, Loss: 45.119871379381145)
Epoch: 200, Loss: 36.14781704423795)
Epoch: 300, Loss: 35.99001703164781)
Epoch: 400, Loss: 35.98724165345199)
Epoch: 500, Loss: 35.987192840248476)
Epoch: 600, Loss: 35.987191981724465)
Epoch: 700, Loss: 35.987191966624785)
Epoch: 800, Loss: 35.98719196635921)
Epoch: 900, Loss: 35.987191966354544)


In [47]:
print(model_simple.betas)
print(model_simple.beta0)

[-6.64073129]
22.283828345335326


In [48]:
mean_squared_error(y_test, predictions)

32.0087666430629

In [49]:
sns.lineplot(x = range(len(history)), y = history);

In [50]:
sns.scatterplot(x = X_test_scl[:, 0], y = y_test )
sns.lineplot(x = X_test_scl[:, 0], y = predictions, color="orange");

Ahora veamos que devuelve `MyGradientDescent`

In [51]:
class MyGradientDescent():
    def __init__(self, learning_rate):
        self.learning_rate = learning_rate
        self.beta1 = 0
        self.beta0 = 0
          
    def fit(self, X, y, epochs = 100):
        N = len(X)
        history = []
        
        for e in range(epochs):
            for i in range(N):
                Xi = X[i, :]
                yi = y.iloc[i] 
                
                hi = self.beta1 * Xi + self.beta0
                f = hi - yi
                
                self.beta1 -= self.learning_rate * 2 / N * f * Xi
                self.beta0 -= self.learning_rate * 2 / N * f 

            loss = 0
            loss = mean_squared_error(y, (self.beta1 * X + self.beta0))
                                      
            if e % 100 == 0:
                print(f"Epoch: {e}, Loss: {loss})")
            
            history.append(loss)
                                      
        return history
                
    def predict(self, X):
        return self.beta1 * X + self.beta0

In [52]:
model_lineal_simple = MyGradientDescent(learning_rate = 0.01)
history = model_lineal_simple.fit(X_train_scl, y_train, 1000)

predictions = model_lineal_simple.predict(X_test_scl)

Epoch: 0, Loss: 555.4547805045436)
Epoch: 100, Loss: 45.50154202643327)
Epoch: 200, Loss: 36.16170328551317)
Epoch: 300, Loss: 35.99042685940659)
Epoch: 400, Loss: 35.987256662664166)
Epoch: 500, Loss: 35.98719402774878)
Epoch: 600, Loss: 35.987192260508245)
Epoch: 700, Loss: 35.987192144208066)
Epoch: 800, Loss: 35.98719213071977)
Epoch: 900, Loss: 35.98719212893573)


In [53]:
print(model_lineal_simple.beta1)
print(model_lineal_simple.beta0)

[-6.64058089]
[22.28345459]


In [54]:
mean_squared_error(y_test, predictions)

32.008910136931455

## Ejercicio 7 - Opcional

Intenten entrenar un modelo con cinco variables predictoras. 

Posiblemente tengan que probar distintos valores de learning rate para conseguir resultados aceptables.

## Conclusión

Implementando de forma más general la clase `MyGradientDescent` logramos usar el mismo código para resolver regresiones lineales simples y múltiples con descenso gradiente.

## Referencias

---

https://towardsdatascience.com/multivariate-linear-regression-in-python-step-by-step-128c2b127171

https://towardsdatascience.com/gradient-descent-in-python-a0d07285742f
